# DSML4220 - Lab 1: Working with Text Data 

In this notebook we'll explore the Airline Tweet dataset and implement the simplest _model_ that we can come up with to predict whether a tweet will be a) positive, b) neutral, or c) negative. To run this notebook in Google Colab or on Kaggle, click one of the following links. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/DSML4220/blob/main/lab1_text_data.ipynb)

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/DSML4220/blob/main/lab1_text_data.ipynb)

When working with Jupyter notebooks you will typically load all of the Python modules/libraries that you need at the beginning in the very first code cell. However, in order for us to see which library we're using where, we'll load them immediately before we need them. (For a brief intro on what Jupyter notebooks are and how they work, check out [this short tutorial by Jeremy Howard](https://www.kaggle.com/code/jhoward/jupyter-notebook-101).)

### 0: Python Review
Before we even get started though, let's quickly review a few important classes/datatypes that will come up often when working with Deep Learning models. The first of these is the [`pandas`](https://pandas.pydata.org/docs/index.html) [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). 

To start, we'll import pandas and create a DataFrame object. But, first, let's create a simple Python [`dictionary`](https://docs.python.org/3/tutorial/datastructures.html#dictionaries):

In [1]:
toy_dict = {'col_a':[1,2,3,4,5], 'col_b':['blue', 'red', 'red', 'purple', 'red']}
toy_dict

{'col_a': [1, 2, 3, 4, 5], 'col_b': ['blue', 'red', 'red', 'purple', 'red']}

Above we created a toy example of a pandas DataFrame. Most often a data frame will be created by reading an input file, but it's also possible to create one manually using a dictionary. We can verify the datatype is `dict` by using the `type()` function:

In [2]:
type(toy_dict)

dict

This dictionary can then be used to create a pandas data frame, where each `dict` key is a column name, and each corresponding `dict` value (which is a list) defines the data in that column.

In [3]:
import pandas as pd

toy_df = pd.DataFrame(toy_dict)
toy_df

,col_a,col_b
0,1,blue
1,2,red
2,3,red
3,4,purple
4,5,red


Now getting back to our problem... we want to use the Airline Tweets (text) dataset to create a simple model for predicting the tweet sentiment. To do so we'll need to load the dataset into a `pandas` `DataFrame`. 

If the data is stored locally then we would use `pd.read_csv("path/to/file/file.csv")` to open it. In this case, the data is online at GitHub. Fortunately, the `pandas` module also knows how to open a file from a URL without any additional parameters. So, we can still use the same method but with the URL instead of the local path, as seen below. After loading the data file, we'll check what the dimensions of the data frame are (i.e. number of rows and number of columns - shown together in a single Python [`tuple`](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)).

In [ ]:
data_URL = 'https://raw.githubusercontent.com/sgeinitz/CS39AA/main/data/trainA.csv'
df = pd.read_csv(data_URL)
df.shape

Now let's look at the first few observations (i.e. tweets) in the data frame using the DataFrame's __head__ method. This is first and most basic step we can do in what is called, [Exploratory Data Analysis (EDA)](https://en.wikipedia.org/wiki/Exploratory_data_analysis). This is a really imporant step for any data science or machine learning project that you'll work on. 

Perhaps not surprising, this is also a step that LLMs (e.g. ChatGPT) have gotten better and better at over the last few months. That is, it's now possible to load an entire .csv file (or other file types) into ChatGPT and ask it to provide some summary statistics and ask it questions about the data. This isn't quite as novel as it may seem though, as there have been Python tools available for some time that to put the EDA phase into a single one-liner (see [ydata-profiling project](https://github.com/ydataai/ydata-profiling)). 

 Nonetheless, it's still vital to know what some of these basic EDA steps look like and to see the data for yourself! Note that you'll often want to use Pandas __describe__ method as well, although for our text data it doesn't do a great job so we are not using it here.

In [ ]:
df.head(10)

To be able to see the the full __text__ field/column we need to tell pandas to change its default column width to be displayed. 

In [ ]:
pd.set_option("display.max_colwidth", 240)
df.head()

Now let's summarize the observations (i.e. tweets) by their labels (i.e. sentiment). We know there should be three possible values for the labels: positive, neutral, and negative, but let's also see how many of each there are. 

In [ ]:
df.sentiment.value_counts(normalize=True)

Let's now see what are the most common words used. Note that this means we need to separate the each tweet into the words. The most basic way to accomplish this is to use Python's string method, __split__. We can first confirm that the _text_ field is in a fact a string by looking at the data frame's data types with __dtypes__. 

In [ ]:
df.dtypes

Note that the _text_ column is not a string data type, but is instead the more general __object__ data type (note that everything in Python inherits from the __object__ data type). Because of Python's friendly dynamic behavior, we don't need to worry about this if we were to look at a single tweet. That is, Python will allow us to use the string.split() method when we are using a single tweet:


In [ ]:
df.iloc[1,1].split()

However, if we try to use split on the entire column we'll run into an issue, as can be seen by the error we encountered here (if you uncomment the line in the following cell).

In [ ]:
#df['text'][:5].split()

To avoid this we need to use the pandas Series __str__ attribute to allow for string methods to be used. Try playing with the following code cell to see what happens when changing the indexing, or including/excluding __.str__ at the end. 

In [ ]:
df['text'][:5].str.split()

Let's now add a new column that contains the list of tokens (i.e. words) for each tweet. Note that the __split()__ method splits a string by the SPACE character by default. This looks alright for now, but we'll see how to improve this later on. Now, however, that we will also convert everything to lowercase before splitting. 

In [ ]:
df['tokens'] = df['text'].str.lower().str.split()
df.head()

Next we will count how often each word occurs across all of the tweets. We'll use a __dict__ for this as well where each __key__ is a word and each word's __value__ will be the number of times it appears in the entire dataset. 

In [ ]:
vocab = dict()
for tweet_tokens in df['tokens']:
    for token in tweet_tokens:
        if token not in vocab:
            vocab[token] = 1
        else:
            vocab[token] += 1

len(vocab)

Let's sort these by the frequency with which each word (i.e. token) appears and then look at the top 20 or so.

In [ ]:
vocab_sorted = dict(sorted(vocab.items(), key=lambda item: item[1], reverse=True))
list(vocab_sorted.items())[:25]

Not very informative, no? Words such as, "_to_", "_the_", "_i_", etc. do not convey a lot information in terms of sentiment. To prevent such uninformative words from influencing the task at hand, most NLP libraries/tasks provide an easy way to remove stop words. 

Before resorting to a Python module designed to work with text data, let's first try continuing with our simple manual approach. Let's see if we can see some difference between in the word (i.e. token) frequencies when we separate the data frame into positive, neutral, and negative. 

In [ ]:
df_pos = df[df['sentiment'] == 'positive']
df_neg = df[df['sentiment'] == 'negative']
df_neu = df[df['sentiment'] == 'neutral']

def create_vocab_list(tokens_column):
    vocab = dict()
    for tweet_tokens in tokens_column:
        for token in tweet_tokens:
            if token not in vocab:
                vocab[token] = 1
            else:
                vocab[token] += 1
    return vocab

vocab_pos = dict(sorted(create_vocab_list(df_pos['tokens']).items(), key=lambda item: item[1], reverse=True))
vocab_neg = dict(sorted(create_vocab_list(df_neg['tokens']).items(), key=lambda item: item[1], reverse=True))
vocab_neu = dict(sorted(create_vocab_list(df_neu['tokens']).items(), key=lambda item: item[1], reverse=True))

In [ ]:
list(vocab_pos.items())[:20]

In [ ]:
list(vocab_neg.items())[:20]

In [ ]:
list(vocab_neu.items())[:20]

Still a lot of stop words in the vocabularies for positive, negative and neutral tweets. Let's try removing the tokens from each of these vocabularies, if the token is also in the top, say, 500, tokens overall. 

In [ ]:
top_n_to_remove = 200 # 100-bad, #500+ better for accuracy
for i, item in enumerate(vocab_sorted.items()):
    if i == top_n_to_remove:
        break
    #print(f" removing token: {item[0]:15} (w/ freq = {item[1]:5}) from vocabs")
    if item[0] in vocab_pos:
        del vocab_pos[item[0]]
    if item[0] in vocab_neg:
        del vocab_neg[item[0]]
    if item[0] in vocab_neu:
        del vocab_neu[item[0]]


In [ ]:
list(vocab_pos.items())[:25]

In [ ]:
list(vocab_neg.items())[:25]

In [ ]:
list(vocab_neu.items())[:25]

That looks a little better! Now, let's try classifying the tweets by looking at one and counting how many tokens it has from the top k tokens in the vocab_pos, vocab_neg, and vocab_neutral sets. Whichever vocab it has the greatest number of tokens from, let's classify it as that. 

To accomplish this let's first create a vocabulary for each of the possible label values. Note that below we are including all the tokens for each label but we could easily include just the top k positive tokens, top k negative, etc.

In [ ]:
classifier_tokens = {"positive": list(vocab_pos.keys())[:], "negative": list(vocab_neg.keys())[:], "neutral": list(vocab_neu.keys())[:]}

Let's count exactly how many tokens will be considered for each class. It's not surprising that there are the most negative tokens. That's because most of the tweets are negative, so there are more unique tokens among the negative tweets. 

In [ ]:
print(f"positive tokens: {len(classifier_tokens['positive'])} \n negative tokens: {len(classifier_tokens['negative'])} \n neutral tokens: {len(classifier_tokens['neutral'])}")

Let's see what the classifier_tokens dictionary looks like


In [ ]:
# let's see the top 20 tokens for each sentiment
for sentiment in classifier_tokens:
    print(f"top 10 tokens for {sentiment} tweets: \n    {classifier_tokens[sentiment][:20]}")

As an example, let's see what happens when we try to classify one single tweet.

In [ ]:
tweet2classify_i = 5557
tweet2classify = df.iloc[tweet2classify_i,:]['tokens']
df.iloc[tweet2classify_i,:]

In [ ]:
pos = 0
neg = 0
neu = 0
for tok in tweet2classify:
    if tok in classifier_tokens['positive']:
        pos += 1
    elif tok in classifier_tokens['negative']:
        neg += 1
    elif tok in classifier_tokens['neutral']:
        neu += 1

print(f"pos: {pos}   neg: {neg}   neu: {neu}")


Here is our simple model. Let's count the number of frequently occurring words. If there are more positive words than negative and netural, then classify it as positive. If there are more neutral than positive or negative, classify neutral. They rest will be classified as negative. NOTE: We could certainly toy around with this more and determine what the default should be, and there should be a threshold different from simply being greater than. 

In [ ]:
def predict_tweet_sentiment(tweet_tokens):
    pos = 0
    neg = 0
    neu = 0
    for tok in tweet_tokens:
        if tok in classifier_tokens['positive']:
            pos += 1
        elif tok in classifier_tokens['negative']:
            neg += 1
        elif tok in classifier_tokens['neutral']:
            neu += 1
    if pos > neg and pos > neu:
        return "positive"
    elif neu > pos and neu > neg:
        return "neutral"
    else:
        return "negative"

Make predictions for all of the tweets in our dataset/dataframe.

In [ ]:
df['predicted_sentiment'] = df['tokens'].apply(lambda x: predict_tweet_sentiment(x))

Let's look at a few of the predictions as compared to the actual (i.e. true) labels. 

In [ ]:
df.head(10)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df['sentiment'], df['predicted_sentiment'])

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix(df['sentiment'], df['predicted_sentiment']), display_labels=['negative', 'neutral', 'positive'])
disp.plot()

In [ ]:
from sklearn.metrics import accuracy_score
mod_accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
print(f"our ad hoc model's accuracy is: {mod_accuracy*100:.2f}%")

Although it's very quick and easy, there are some areas in which we can improve upon the approach we used above. 

1. __Better Tokenization:__ The tokenization is very crude right now. Simply splitting a tweet into tokens with the 'SPACE' character as a delimiter means that positive tokens like _"awesome."_ and _"awesome!"_ are considered as two different tokens. Cases such as that one are relatively easy to solve since it just meaning removing some punctuation. However, different forms of a word, such as _"delay"_ and _"delayed"_ can cause some issues too. For our ad hoc modeling approach above, resolving these tokenization don't seem to affect the outcome too much right now. But, once we try to use a proper model (e.g. naive bayes, neural network, etc.), then these tokenization deficiences can affect model performance even more. For example, having all of the different forms of the verb _"delay"_ will mean that our model needs to have that many more parameters in it. 
2. __Vectorization__: Even with better tokenization, we still need to do more modify the data to be able to use other types of models. What we specifically need to do is convert the tokens into a numerical representation of some kind. When working text data, this process of converting text/tokens into a numerical representation will allow us to use many different types of models, including neural networks. 
3. __Modeling Process/Evaluation:__ The modeling process and assessment need to be improved. To start, our accuracy of {{mod_accuracy}} is not much better than if we simply label every tweet as negative (since 65% of all tweets are negative). So we need to be sure that we're comparing our model performance metrics to a suitable baseline. The larger issue, however, is that we don't have a validation and/or test set right now. We used all 10k observations to build our positive, negative, and neutral sets of words; then we check our accuracy on these exact same sets of tweets. To really understand how our model will work for a new tweet that we have never seen, and that is posted in the future, we need to remove a portion of the dataset from the 'training', then evaluate our model against this. 

We'll come back to this dataset in one or notebooks and address these, but for now try to think about what impact they are having and how best we can resolve them. 

